In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import os
from urllib.parse import urlparse
import glob
from pathlib import Path

In [3]:
iteration = 1
day = 1;
how_many = 365
year = 2021

In [ ]:
while iteration <= how_many:
    
    def get_url_paths(url, ext='', params={}):
        response = requests.get(url, params=params)
        if response.ok:
            response_text = response.text
        else:
            return response.raise_for_status()
        soup = BeautifulSoup(response_text, 'html.parser')
        parent = [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]
        return parent

    if day < 10:
        url = 'https://spdf.gsfc.nasa.gov/pub/data/gold/level1c/' + str(year) + '/00' + str(day) + "/"
    if day >= 10 and day < 100:
        url = 'https://spdf.gsfc.nasa.gov/pub/data/gold/level1c/' + str(year) + '/0' + str(day) + "/"
    if day >= 100:
        url = 'https://spdf.gsfc.nasa.gov/pub/data/gold/level1c/' + str(year) + '/' + str(day) + "/"
    ext = 'nc'
    result = get_url_paths(url, ext)
    
    result = [k for k in result if 'NI1' in k]
    
    for f in result:
        a = urlparse(f)
        name = os.path.basename(a.path)
        testfile = urllib.request.URLopener()
        testfile.retrieve(f, name)
    day = day + 1;
    iteration = iteration + 1;
    
    files = glob.glob(r'*.nc')

    for f in files:
        day = f[22:25]
        Path("data/" + str(year) + "/" + day).mkdir(parents=True, exist_ok=True)
        Path(f).rename("data/" + str(year) + "/" + day + "/" + f)


In [ ]:
files = glob.glob(r'data/' + str(year) + '/*/*.nc')

for f in files:
    os.rename(f, f[0:45] + ".nc")